# Extract ES cert from docker container
docker cp gdelt_elk-es01-1:/usr/share/elasticsearch/config/certs/ca/ca.crt ./ca.crt

In [ ]:
from elasticsearch import Elasticsearch
from datetime import datetime
import os
CA_CERT_PATH = os.path.abspath("ca.crt")  # Converts to full path

# --- CONFIG ---
ES_HOST = "https://localhost:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "changeme"
ES_INDEX = "gkg"       

# Create the client with authentication
es = Elasticsearch(
    ES_HOST,
    basic_auth=(ES_USERNAME, ES_PASSWORD),
    verify_certs=True, # Set to False if you're using self-signed certs (not recommended for prod)
    ca_certs=CA_CERT_PATH
)

In [12]:
# --- INPUT ---
start_time = "2025-03-25 06:30:00"
end_time = "2025-03-25 06:31:00"

# --- Convert to RecordId format ---
def to_record_id_format(dt_str):
    return datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M%S")

record_id_start = to_record_id_format(start_time)
record_id_end = to_record_id_format(end_time)

# --- Query ---
query = {
    "_source": ["V2Persons.V1Person"],
    "query": {
        "range": {
            "RecordId": {
                "gte": record_id_start,
                "lte": record_id_end
            }
        }
    }
}

# --- Execute ---
response = es.search(index=ES_INDEX, body=query)  # Increase size if needed

# --- Extract and Deduplicate Persons ---
entities = []
for hit in response["hits"]["hits"]:
    persons = hit["_source"].get("V2Persons", {}).get("V1Person", [])
    entities.extend(persons)

unique_entities = sorted(set(entities))

# --- Output ---
print(f"Entities found from {start_time} to {end_time}:")
for name in unique_entities:
    print("-", name)


/home/otb-02/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Entities found from 2025-03-25 06:30:00 to 2025-03-25 06:31:00:
- Alexandra Heller-Nicholas
- Arvind Kumar
- Barry Jenkin
- Basel Adra
- Bob Dylan
- Chris Lacivita
- Conan Obrien
- Donald Trump
- Elon Musk
- Gerard Depardieu
- Gregory Maguire
- Gurpreet Singh
- Hamdan Ballal
- Holden Sheppard
- Jack Mccollough
- John Howard
- Jonathan Anderson
- Josh Kimelman
- Julia Gillard
- Jupally Krishna Rao
- Kevin Rudd
- Lazaro Hernandez
- Lea Tsemel
- Los Angeles
- Maggie Haberman
- Malcolm Turnbull
- Mark Zuckerberg
- Masafar Yatta
- Masafer Yatta
- Parikshit Mehra
- Peter Baker
- Ponguleti Srinivas Reddy
- Pres Fernanda Figueroa
- Proenza Schouler
- Rachel Szor
- Revanth Reddy
- Santhi Kumari
- Sarah Silverman
- Seth Rogen
- Shankar Loteti
- Sophie Cunningham
- Surendra Singh
- Telangana Revanth Reddy
- Tracey Spicer
- Yuval Abraham
